# Ipyvolume snapshots

An experiment with taking a list of snapshots of the camera position (not the shown data) for later reuse. That state could later be dumped to and loaded from an external storage using e.g. the `pickle` module and/or shown with a thumbnail using `Figure.screenshot()` (?). Also, the UI could be a play button or dropdown menu instead, depending on context.

In [1]:
import numpy as np
import ipyvolume as ipv
from ipywidgets import Button, HBox, IntSlider, Layout, VBox

In [2]:
class SnapshotMgr:
    """This class maintains a list of snapshots for an ipyvolume figure.
    
    A snapshot is the state of the camera position and its world context,
    but does not comprise any data shown in the figure itself.
    """
    def __init__(self, figure=None):
        """Constructor.
        """
        assert figure
        
        self.history = []
        self.fig = figure

        self.slider = IntSlider(description='Snapshots:',
                                min=0, max=max(0, len(self.history)-1))
        self.slider.observe(self.recall_state, names=["value"])
        self.slider.disabled = len(self.history) <= 1

        layout = Layout(width="50px")
        self.add_button = Button(tooltip="Add snapshot to history", icon="plus",
                                 layout=layout, button_style="primary")
        self.add_button.on_click(self.add_state)
        self.rm_button = Button(tooltip="Drop snapshot from history", icon="trash-o",
                                layout=layout, button_style="primary", disabled=True)
        self.rm_button.on_click(self.drop_state)
        self.history.append(self.get_figure_state())
        self.ui = VBox([self.fig, HBox([self.slider, self.add_button, self.rm_button])])

    def get_figure_state(self):
        """Get state of a figure and return as a dictionary.
        """
        fig = self.fig
        return dict(
            position = fig.camera.position,
            projectionMatrix = fig.camera.projectionMatrix,
            rotation = fig.camera.rotation,
            quaternion = fig.camera.quaternion,
            matrix_world = fig.matrix_world,
            matrix_projection = fig.matrix_projection,
            xlim = fig.xlim,
            ylim = fig.ylim,
            zlim = fig.zlim
        )

    def restore_figure_state(self, state):
        """Restore figure to another state.
        """
        fig = self.fig
        fig.camera.position = state["position"]
        fig.camera.projectionMatrix = state["projectionMatrix"]
        fig.camera.rotation = state["rotation"]
        fig.camera.quaternion = state["quaternion"]
        fig.matrix_world = state["matrix_world"]
        fig.matrix_projection = state["matrix_projection"]
        fig.xlim = state["xlim"]
        fig.ylim = state["ylim"]
        fig.zlim = state["zlim"]

    def add_state(self, event):
        """Add current state to the list.
        """
        h = self.history
        s = self.slider
        if self.fig:
            h.append(self.get_figure_state())
            s.max = len(h) - 1
            s.value = s.max
            self.rm_button.disabled = len(h) <= 1
            s.disabled = len(h) < 1

    def drop_state(self, event):
        """Drop current state from the list.
        """
        h = self.history
        s = self.slider
        if len(h) > 0:
            del h[s.value]
            if s.value > 0:
                s.value -= 1
            s.max = max(0, len(h) - 1)
            self.rm_button.disabled = len(h) <= 1
            s.disabled = len(h) < 1
            self.restore_figure_state(h[s.value])

    def recall_state(self, event):
        """Retrieve previous state from the list.
        """
        self.restore_figure_state(self.history[event.new])

In [3]:
fig = ipv.figure()
x, y, z = np.random.normal(0, 1, (3, 10_000))
scatter = ipv.scatter(x, y, z, size=1, marker="sphere")
mgr = SnapshotMgr(figure=fig)
mgr.ui